<a href="https://colab.research.google.com/github/anshupandey/AIOps-EL/blob/main/Part3B_Inferense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferense with mlflow model registry

In [1]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.6 MB/s eta 0:00:00


In [2]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import numpy as np

In [6]:
mlflow.set_tracking_uri("http://3.107.14.172:5000")
import os
os.environ['MLFLOW_TRACKING_URI'] = "http://3.107.14.172:5000"

In [10]:
model_name = "Anshu-Latency-Pred2"
alias = "production"

model = mlflow.pyfunc.load_model(f"models:/{model_name}@{alias}")

In [19]:
model.metadata.get_input_schema()

['Application_Type': string (required), 'Signal_Strength': double (required), 'Required_Bandwidth': double (required), 'Allocated_Bandwidth': double (required), 'Signal_Quality_Category': string (optional)]

In [37]:
data = pd.DataFrame({"Application_Type":['Web_Browsing',],
                     "Signal_Strength":[-88.0],
                     "Required_Bandwidth":[1.0],
                     "Allocated_Bandwidth":[1.0],
                     "Signal_Quality_Category":['weak']})

model.predict(data)

array([21.97613595])

## Logging Inferense Pipeline

In [21]:
import shap

In [22]:
mlflow.set_experiment("Latency_inferense")

2024/12/11 03:27:03 INFO mlflow.tracking.fluent: Experiment with name 'Latency_inferense' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/262032827791439357', creation_time=1733887623558, experiment_id='262032827791439357', last_update_time=1733887623558, lifecycle_stage='active', name='Latency_inferense', tags={}>

In [24]:
model_name = "Anshu-Latency-Pred2"
alias = "production"

model = mlflow.pyfunc.load_model(f"models:/{model_name}@{alias}")

In [42]:
model2 = model.get_raw_model()
model2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Signal_Strength',
                                                   'Required_Bandwidth',
                                                   'Allocated_Bandwidth']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Application_Type',
                                                   'Signal_Quality_Category'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=12, min_samples_leaf=3,
                                       random_state=42))])

In [45]:
preprocessor = model2.named_steps['preprocessor']
data2 = preprocessor.transform(data)

In [47]:
mlflow.autolog()
with mlflow.start_run(log_system_metrics=True) as run:
  mlflow.shap.log_explanation(model2.named_steps['regressor'].predict,data2)

2024/12/11 03:44:58 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/12/11 03:44:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/12/11 03:44:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2024/12/11 03:44:59 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2024/12/11 03:44:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2024/12/11 03:45:00 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/11 03:45:00 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


  0%|          | 0/1 [00:00<?, ?it/s]

🏃 View run receptive-mole-899 at: http://3.107.14.172:5000/#/experiments/262032827791439357/runs/f349d187fec4457f92665104654f3d6f
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/262032827791439357


2024/12/11 03:45:03 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/11 03:45:04 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
